In [1]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install --upgrade datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import wandb
wandb.login()

wandb: Currently logged in as: minooshayan97 (minoosh). Use `wandb login --relogin` to force relogin


True

In [5]:
# finetune

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
from datasets import load_dataset

dataset = load_dataset("minoosh/shEMO", use_auth_token=True)

  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'emotion'],
        num_rows: 2400
    })
    test: Dataset({
        features: ['audio', 'emotion'],
        num_rows: 300
    })
    valid: Dataset({
        features: ['audio', 'emotion'],
        num_rows: 300
    })
})

In [9]:
dataset['valid']

Dataset({
    features: ['audio', 'emotion'],
    num_rows: 300
})

In [10]:
from datasets import load_metric

metric = load_metric("accuracy")

<ipython-input-10-350fcae91df9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [11]:
labels = dataset["train"].features["emotion"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["2"]

'N'

In [12]:
model_checkpoint = "minoosh/wav2vec2-base-finetuned-ie"

In [13]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [14]:
max_duration = 5.0

In [15]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

In [16]:
encoded_ds = dataset.map(preprocess_function, remove_columns=["audio"], batched=True)
encoded_ds

DatasetDict({
    train: Dataset({
        features: ['emotion', 'input_values'],
        num_rows: 2400
    })
    test: Dataset({
        features: ['emotion', 'input_values'],
        num_rows: 300
    })
    valid: Dataset({
        features: ['emotion', 'input_values'],
        num_rows: 300
    })
})

In [17]:
encoded_ds = encoded_ds.rename_column("emotion", "label")

In [18]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at minoosh/wav2vec2-base-finetuned-ie and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([4, 256]) in the checkpoint and torch.Size([6, 256]) in the model instantiated
- classifier.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
%env WANDB_PROJECT = shEMO_SER

env: WANDB_PROJECT=shEMO_SER


In [20]:
model_name = 'iewav2vec2'
batch_size = 8
args = TrainingArguments(
    f"{model_name}-finetuned-on-shEMO",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    report_to='wandb',
    run_name='finetune_iewav2vec2_on_shEMO'
)

In [21]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [22]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_ds['train'],
    eval_dataset=encoded_ds['valid'],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

/content/iewav2vec2-finetuned-on-shEMO is already a clone of https://huggingface.co/minoosh/iewav2vec2-finetuned-on-shEMO. Make sure you pull the latest changes with `repo.git_pull()`.


In [23]:
import torch
torch.cuda.empty_cache()

In [24]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.077900,0.973570,0.713333
2,0.775300,0.786850,0.746667
3,0.578400,0.657896,0.786667
4,0.508800,0.639305,0.793333
5,0.445400,0.608983,0.830000
6,0.319700,0.773524,0.806667
7,0.235000,0.694263,0.833333
8,0.164800,0.695782,0.836667
9,0.101600,0.743715,0.833333
10,0.082200,0.643524,0.846667


TrainOutput(global_step=1500, training_loss=0.2458931110824148, metrics={'train_runtime': 5314.3372, 'train_samples_per_second': 9.032, 'train_steps_per_second': 0.282, 'total_flos': 2.1475813401980895e+18, 'train_loss': 0.2458931110824148, 'epoch': 20.0})

In [25]:
#trainer.train(resume_from_checkpoint=True )

In [26]:
trainer.evaluate(encoded_ds['test'])

{'eval_loss': 0.9088177680969238,
 'eval_accuracy': 0.82,
 'eval_runtime': 15.9497,
 'eval_samples_per_second': 18.809,
 'eval_steps_per_second': 2.382,
 'epoch': 20.0}

In [27]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
To https://huggingface.co/minoosh/iewav2vec2-finetuned-on-shEMO
   30ade68..8d7b8a2  main -> main

   30ade68..8d7b8a2  main -> main

To https://huggingface.co/minoosh/iewav2vec2-finetuned-on-shEMO
   8d7b8a2..3cc6a56  main -> main

   8d7b8a2..3cc6a56  main -> main



'https://huggingface.co/minoosh/iewav2vec2-finetuned-on-shEMO/commit/8d7b8a2f04e0bdf8d79ad3f479f30a47dd254080'